In [ ]:
# 04/14/2025 

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 51
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

τ = ones(N*J, N) # Iceberg trade costs
Lt = ones(N*J, periods) # Size of labor force in each country at time 


Ldot = ones(N*J, periods)

At0 = ones(N*J)#initial productivities


wt = ones(N*J, periods)
wt0 = ones(N*J)

wdot = ones(N*J, periods)
tradesharest0 = ones(N*J, N) * (1 / N)
Adot = ones(N*J, periods)


kdot = ones(N*J, N, periods)

pdotArray = ones(N*J, periods)
d1wdot = ones(N*J)

μtminus1 = zeros(N*J, N*J) #value of μ at time t = -1
πt0 = ones(N*J, N) #initial trade shares
μt = ones(N*J,N*J,periods)*(1/(N*J))

# a guess for path of udot:
udotPathGuess = ones(N*J, periods)
udotPathUpdate = zeros(N*J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm

function θindex(j)
    div(j-1, N) + 1  # Find the correct index for θ
end

function pdot(n, d1wdot, kdot, Adot, time, tradesharest0)
    j = θindex(n)
    (sum(tradesharest0[n, i] * (d1wdot[(j-1) * N + i] * kdot[n, i, time])^-θ[j] * Adot[(j-1) * N + i, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end ##### Going to need to update tradeshares

function tradeSharest0(n, i, wt0, At0, τ)
    j = θindex(n)
    wt0[(j-1) * N + i] * τ[n,i]^-θ[j] * At0[(j-1) * N + i] ^θ[j] / (sum((wt0[(j-1) * N + m] *τ[n,m])^-θ[j] * At0[(j-1) * N + m]^θ[j] for m in 1:N))
end



function tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time, tradesharest0)
    j = θindex(n)
    tradesharest0[n, i] * ((d1wdot[(j-1) * N + i] * kdot[n, i, time]) / pdot(n, d1wdot, kdot, Adot, time, tradesharest0))^-θ[j] * Adot[(j-1) * N + i, time]^θ[j]
end

function incomet0(n, wt0, Lt) #income of country-sector in time t=0 given wage and labor  
    wt0[n] * Lt[n, 1] 
end

function incomet1(n, d1wdot, time, Lt, wt)
    wt[n, time]*(d1wdot[n])*Lt[n, time]*(Ldot[n, time])
end

function Xt0(n, α, wt0, Lt) #expenditure on sector good j in region n
    j = θindex(n)
    country_index = mod(n - 1, N) + 1
    α[j] * sum(wt0[country_index + N*(k-1)] * Lt[country_index + N*(k-1), 1] for k in 1:J) #from equation (8)
end

function Xt1(n, α, d1wdot, Lt, wt, Ldot, time)
    j = θindex(n)
    country_index = mod(n - 1, N) + 1
    α[j] * sum(d1wdot[country_index + N*(k-1)] * Ldot[country_index + N*(k-1), time] * wt[country_index + N*(k-1), time] * Lt[country_index + N*(k-1), time] for k in 1:J) #from equation (14) 
end


while errormax > .01 
    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N*J
                μt[n,i,time+1] = (μt[n,i,time]*(udotPathGuess[i,time+1])^(β/v))/sum(μt[n,h,time]*(udotPathGuess[h,time+1])^(β/v) for h in 1:N*J)
            end
        end
        for n in 1:N*J
            Lt[n, time + 1] = sum(μt[i, n, time] * Lt[i, time] for i in 1:N*J) ### Step 3 of the appendix D
        end

        Ldot[:, time] .= Lt[:,time+1]./Lt[:,time] # updates Ldot 
    end

    function g!(G, wt0)
        for n in 1:N*J
            j = div(n-1, N) + 1  # Find the correct index for j
            country = mod(n - 1, N) + 1
            #################FIXME
            #G[n] = incomet0(n, wt0, Lt) - sum(Xt0((j-1) * N + i, α, wt0, Lt) * tradesharest0[(j-1) * N + i, country] for i in 1:N) #market clearing for wt0
            G[n] = incomet0(n, wt0, Lt) - sum(Xt0((j-1) * N + i, α, wt0, Lt) * tradeSharest0((j-1) * N + i, country, wt0, At0, τ) for i in 1:N) #market clearing for wt0



        end
    end
    results = nlsolve(g!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages
    
    wt0[:] .= results.zero[:]
    wt[:, 1] .= wt0[:]
    for n in 1:N*J
        for i in 1:N
            tradesharest0[n, i] = tradeSharest0(n, i, wt0, At0, τ)
        end
    end

    println(wt0)  #end solving for wt0

    for time in 1:periods-1

        function f!(F, d1wdot)
            for n in 1:N*J
                j = div(n-1, N) + 1  # Find the correct index for j
                country = mod(n - 1, N) + 1

                F[n] = (incomet1(n, d1wdot, time, Lt, wt) - sum(tradeSharest1((j-1) * N + i, country, d1wdot, Ldot, Adot, kdot, time, tradesharest0)
                    * Xt1((j-1) * N + i, α, d1wdot, Lt, wt, Ldot, time) for i in 1:N)) # equation (15)
            end
        end
        println("checkpoint")
        results = nlsolve(f!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages with country 1 set at 1.0
        sol = ones(N*J, periods)

        wdot[:, time] .= results.zero[:] # updates wdot with solution to the system of equations
        d1wdot[:] .= wdot[:,time] 

        for i = 1:N*J
            println(wdot[i, time])
        end

        println("checkpoint")
        
        for n in 1:N
            pdotArray[n,time] = pdot(n, wdot, kdot, Adot,time, tradesharest0)
        end
    
        tradesharest0[:,:] .= [tradeSharest1(n,i,d1wdot,Ldot,Adot,kdot,time, tradesharest0) for n in 1:N*J, i in 1:N]

        wt[:, time + 1] .= wdot[:, time] .* wt[:, time]

    end

    for time in 1:periods-1
        for n in 1:N*J
            udotPathUpdate[n,time] = wdot[n,time]*(sum(μt[n,i,time]*(udotPathGuess[i,time+1])^(β/v) for i in 1:N*J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end

[1.2364493990555114, 1.2364493990540932, 1.236449399055056, 1.2364493990536443, 1.2364493990544476, 1.2364493990542436]
checkpoint
1.0020537251465838
1.0020537251468138
1.0020537251469512
1.0020537251471802
1.0020537251470534
1.0020537251470865
checkpoint
checkpoint
1.098741741440803
1.0987417414408034
1.0987417414407938
1.0987417414407947
1.0987417414408032
1.0987417414408018
checkpoint
checkpoint
1.116267996538603
1.1162679965386024
1.1162679965386115
1.1162679965386104
1.1162679965386013
1.1162679965386029
checkpoint
checkpoint
1.1123835337240342
1.1123835337240342
1.1123835337240342
1.1123835337240344
1.1123835337240346
1.1123835337240346
checkpoint
checkpoint
1.0679776258462566
1.0679776258462566
1.0679776258462579
1.0679776258462577
1.0679776258462572
1.067977625846257
checkpoint
checkpoint
2.2848469944203456
2.284846994420345
2.2848469944203402
2.28484699442034
2.284846994420344
2.2848469944203442
checkpoint
WHILE LOOP
[1.2232214390266105, 1.2232214390249752, 1.2232214390267664,

0.10999097539763325

0.0

In [24]:
#2/28/2025 with d1wdot instead of wdot
using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 7
J = 3 # Number of goods (needs to be big number and an integer type) 
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)
T = ones(N*J, periods) # Frechet scale parameter (governs absolute advantage)
σ = 2 # Substitution elasticity between goods
τ = ones(N*J, N, periods) # Iceberg trade costs
L = ones(N*J, periods) # Size of labor force in each country
Ldot = ones(N*J, periods)


wt = ones(N*J, periods)
Lt = ones(N*J, periods)
wdot = ones(N*J, periods)
tradesharest0 = ones(N*J, N) * (1 / N)
Adot = ones(N*J, periods)

kdot = ones(N*J, N, periods)
TSt1 = ones(N*J, N, periods) 
pdotArray = ones(N*J, periods)
d1wdot = ones(N*J)
μ = zeros(N*J, N*J, periods)

μtminus1 = ones(N*J, N*J) #value of μ at time t = -1
πt0 = ones(N*J, N) #initial trade shares
wLt0 = ones(N*J, N) #initial values of w*L
rHt0 = ones(N*J, N) #initial values of r*H
μt = zeros(N*J,N*J,periods)

# a guess for path of udot:
udotPathGuess = ones(N*J, periods)
udotPathUpdate = zeros(N*J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm

function θindex(j)
    div(j-1, N) + 1  # Find the correct index for θ
end

function pdot(n, d1wdot, kdot, Adot, time)
    j = θindex(n)
    (sum(tradesharest0[n, i] * (d1wdot[(j-1) * N + i] * kdot[n, i, time])^-θ[j] * Adot[(j-1) * N + i, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end ##### Going to need to update tradeshares

function tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
    j = θindex(n)
    tradesharest0[n, i] * ((d1wdot[(j-1) * N + i] * kdot[n, i, time]) / pdot(n, d1wdot, kdot, Adot, time))^-θ[j] * Adot[(j-1) * N + i, time]^θ[j]
end
function incomet1(n, d1wdot, time)
    wt[n, time]*(d1wdot[n])*L[n, time]*(Ldot[n, time])
end
function Xt1(n, α, d1wdot, tradesharest0, Ldot, Adot, kdot, time)
    sum(incomet1(n, d1wdot, time) * tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time) for i in 1:N)
end



while errormax > .01 
    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N*J
                if time == 1
                    μt[n,i,time] = μt[n,i,time] + (μtminus1[n,i]*udotPathGuess[i,time+1])/sum(μtminus1[n,i]*udotPathGuess[h,time+1] for h in 1:N*J)
                else 
                    μt[n,i,time] = μt[n,i,time] + (μt[n,i,time-1]*udotPathGuess[i,time+1])/sum(μt[n,h,time-1]*udotPathGuess[h,time+1] for h in 1:N*J)
                end
            end
        end
        for i in 1:N*J ### Step 3 of the appendix D
            for j in 1:N*J
                Lt[i ,time + 1] = Lt[i ,time + 1] + Lt[j, time]*μt[i,j,time]
            end
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N
                TSt1[n, i, time] = tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
            end
        end
        #############################################
        function f!(F, d1wdot)
            for n in 1:N*J
                j = div(n-1, N) + 1  # Find the correct index for j
                country = n % N 
                if country == 0
                    country = N
                end
                #F[n] = (d1wdot[n] * Ldot[n,time] * wt[n,time] * Lt[n,time] - sum(tradeSharest1((i-1) * J + j, country, d1wdot, Ldot, Adot, kdot, time)
                #    * Xt1((i-1) * J + j, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N)) # Wrong indexing!!!
                F[n] = (incomet1(n, d1wdot, time) - sum(tradeSharest1((j-1) * N + i, country, d1wdot, Ldot, Adot, kdot, time)
                    * Xt1((j-1) * N + i, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N))
            end
        end
        println("checkpoint")
        results = nlsolve(f!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages with country 1 set at 1.0
        sol = ones(N*J, periods)
        for i = 1:N*J
            sol[i, time] = results.zero[i] #saving results in sol[i]
            wdot[i, time] = sol[i,time]
            println(sol[i,time])
            wdot[i, time] = sol[i, time]
        end
        println("checkpoint")
        for n in 1:N
            pdotArray[n,time] = pdot(n, wdot, kdot, Adot,time)
        end
    
        for n in 1:N*J 
            for i in 1:N
                tradesharest0[n,i] = TSt1[n,i,time]
            end
        end
        for i in 1:N*J
            wt[i, time + 1] = wdot[i,time]*wt[i,time]
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            udotPathUpdate[n,time] = wt[n,time]*(sum(μt[n,i,time]*udotPathGuess[i,time+1] for i in N*J)) ##needs verification but is equation 17 as specified by step 5
        end ## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end

checkpoint


BoundsError: BoundsError: attempt to access 6×2×7 Array{Float64, 3} at index [1, 1]

In [ ]:

import Pkg; Pkg.add("SpecialFunctions")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [276daf66] + SpecialFunctions v2.5.0
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
